In [35]:
import os
import glob as gb
import pandas as pd
import logging
from datetime import datetime
import json
import xml.etree.ElementTree as et

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='newlog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)


def extract_csvdata():
    try:
        csvfiles=gb.glob('C:/Users/User/source/*.csv')
        CsvDf=[]
        for files in csvfiles:
            Df=pd.read_csv(files)
            CsvDf.append(Df)

        Finaldf=pd.concat(CsvDf,ignore_index=True)
        logger.info('Data extracted from csv files')
        return Finaldf
        
    except Exception as e:
        logger.error(f'Error in Extracting data from csv files:{e}')
        raise

def extract_Jsondata():
    try:
        jsondf=[]
        Jfiles=gb.glob('C:/Users/User/source/*.json')
        for file in Jfiles:
            Df=pd.read_json(file,lines=True)
            jsondf.append(Df)

        finaljsondf=pd.concat(jsondf,ignore_index=True)
        logger.info('Data extracted from json files')
        return(finaljsondf)

    except Exception as e:
        logger.error(f'Error in Extracting data from json files:{e}')
        raise

def extract_xmldata():
        try:
            xmlfiles = gb.glob('C:/Users/User/source/*.xml')
            xmldf = []
            for file in xmlfiles:
                tree = et.parse(file)
                root = tree.getroot()
                data = []
                for elem in root:
                        record = {}
                        for child in elem:
                                record[child.tag] = child.text
                        data.append(record)
                Df = pd.DataFrame(data)
                xmldf.append(Df)
            finalxmldf= pd.concat(xmldf, ignore_index=True)
            return finalxmldf
        except Exception as e:
            logger.error(f'Error in Extracting data from xml files:{e}')
            raise
        

def Combine_dfs():
     try:
        csv_df=extract_csvdata()
        json_df=extract_Jsondata()
        xml_df=extract_xmldata()
        sdf=[csv_df,json_df,xml_df]
        cdf=pd.concat(sdf,ignore_index=True)
        logger.info('All data frames combined successfully')
        return cdf
     except Exception as e:
        logger.error(f'Error in combining all data farmes:{e}')
        raise
     
        

def transform_data(Df):
    try:
        Df['height'] = pd.to_numeric(Df['height'])
        Df['HeightsinMeters']=Df['height']*0.3048
        Df['weight'] = pd.to_numeric(Df['weight'])
        Df['WeightsinKgs']=Df['weight']*0.453592
        logger.info('Data transformed successfully')
        return(Df)
    except Exception as e:
        logger.error(f'Error transforming data :{e}')
        raise

def load_data(Df,combinedfile1):
    try:
        Df.to_csv(combinedfile1,index=False)
        logger.info('Data Loading is successful')
    except Exception as e:
        logger.error(f'Error in Loading data :{e}')
        raise

def etl_process():
   # try:

        #input_path='C:/Users/User/source'
        Output_path='C:/Users/User/source/combinedfile.csv'
        data =Combine_dfs()
        T_data=transform_data(data)
        load_data(T_data,Output_path)
        logger.info('ETL process completed successfully')
    #except Exception as e:
       # logger.error(f'ETL process failed :{e}')
        #raise

if __name__=='__main__':
    etl_process()
